### 🌵 Parastichy Data Collection Instructions

Follow these steps to trace the spiral patterns (parastichies) on your plant image.

**Before you start:** Click the **Play button (▶)** on the left side of the code cell below. Then, scroll to the bottom of the cell to interact with the tool.

---

1. **Upload & Info:**
   - **Upload your image:** Click the upload button. `.jpg` or `.png` are preferred, but most formats work.
   - **Enter your details:** Provide your **Full Name** (for publication credit), **Institution** (or `none`), **Location** (e.g., "El Charco botanical garden" or "UNAM Jardín Botánico), and a valid **Email Address**.
   - Click **START TRACING** to load your image.

2. **Tracing Family A:**
   - Click points along a single spiral "arm," moving from the **center to the periphery**. Try to click every sequential node (areole/tubercle, leaf, etc) you can identify.
   - When one arm is finished, click **NEXT ARM**.
   - Trace **every arm** visible in that specific direction.
   - **The Guardrail:** The total number of arms in this family must be a Fibonacci number (e.g., `5`, `8`, `13`, `21`, `34`, etc.). If the count is incorrect, the tool will alert you.
   - When finished with the first set, click **FINISH FAMILY A**.

3. **Tracing Family B:**
   - You will now see your first family in **black**. Trace the arms spiraling in the **opposite direction**.
   - **Tip:** When a point in Family B crosses a black point from Family A, click as close to the black point as possible. This helps us consolidate the data.
   - Click **NEXT ARM** after each arm, and **FINISH & DOWNLOAD ZIP** when the entire second family is complete.
   - **The Guardrail:** Family B must be the Fibonacci number immediately adjacent to Family A (e.g., if Family A was `13`, Family B must be `8` or `21`).

4. **Your Downloaded Results:**
   The tool will automatically download a `.zip` file named after your image. Inside you will find:
   - 🖼️ **The original image** and a **plotted version** (`_parastichies`) showing your work.
   - 📊 **Results CSV:** A data file containing every coordinate you clicked.
   - 📝 **Info CSV:** A file containing your authorship and location information.

**Final Step:** Please deposit your `.zip` file or email it to the project lead as instructed. Your contribution is vital to our collective analysis!


In [ ]:
import os, base64, json, zipfile, numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from skimage import io
from google.colab import files, output
from IPython.display import HTML, display

# --- 1. UPLOAD IMAGE ---
print("--- STEP 1: UPLOAD IMAGE ---")
uploaded = files.upload()
if not uploaded:
    raise Exception("No file uploaded.")
image_filename = list(uploaded.keys())[0]
image = io.imread(image_filename)
img_ext = os.path.splitext(image_filename)[1]

with open(image_filename, "rb") as f:
    img_base64 = base64.b64encode(f.read()).decode('utf-8')

# --- 2. BACKEND STORAGE ---
def finalize_collection(json_payload):
    payload = json.loads(json_payload)
    print("\n--- GENERATING FINAL OUTPUTS ---")
    clean_name = os.path.splitext(image_filename)[0]

    # Generate CSVs
    csv_rows = []
    for fam in ["A", "B"]:
        for arm_idx, arm in enumerate(payload[f'family{fam}']):
            for pt_idx, pt in enumerate(arm):
                csv_rows.append({"Family": fam, "Arm_ID": arm_idx + 1, "Point_ID": pt_idx + 1,
                                 "X_pixel": round(pt[0], 2), "Y_pixel": round(pt[1], 2)})

    results_csv = f"{clean_name}_results.csv"
    pd.DataFrame(csv_rows).to_csv(results_csv, index=False)
    info_csv = f"{clean_name}_info.csv"
    pd.DataFrame([payload['info']]).to_csv(info_csv, index=False)

    # Generate Plot
    plot_filename = f"{clean_name}_parastichies{img_ext}"
    fig, ax = plt.subplots(figsize=(12, 12))
    ax.imshow(image)
    for arm in payload['familyA']:
        pts = np.array(arm)
        ax.plot(pts[:, 0], pts[:, 1], 'o-', color='cyan', markersize=4, linewidth=1)
    for arm in payload['familyB']:
        pts = np.array(arm)
        ax.plot(pts[:, 0], pts[:, 1], 'o-', color='magenta', markersize=4, linewidth=1)
    ax.axis('off')
    plt.savefig(plot_filename, bbox_inches='tight', pad_inches=0, dpi=150)
    plt.close()

    # Create ZIP
    zip_path = f"{clean_name}_package.zip"
    with zipfile.ZipFile(zip_path, 'w') as zipf:
        zipf.write(results_csv); zipf.write(info_csv); zipf.write(plot_filename); zipf.write(image_filename)

    print(f"Success! Captured {len(payload['familyA'])} and {len(payload['familyB'])} arms.")
    files.download(zip_path)

output.register_callback('notebook.FinalizeCollection', finalize_collection)

# --- 3. THE INTERFACE ---
html_code = f"""
<div id="ui-container" style="background:#f4f4f9; padding:25px; border-radius:15px; font-family:sans-serif; color:#333;">
    <div id="info-form">
        <h2>Phyllotaxis Data Collector</h2>
        <input type="text" id="name" placeholder="Name" style="width:90%; padding:10px; margin-bottom:10px;"><br>
        <input type="text" id="inst" placeholder="Institution" style="width:90%; padding:10px; margin-bottom:10px;"><br>
        <input type="text" id="loc" placeholder="Photo Location" style="width:90%; padding:10px; margin-bottom:10px;"><br>
        <input type="email" id="email" placeholder="Email" style="width:90%; padding:10px; margin-bottom:20px;"><br>
        <button onclick="startTracing()" style="padding:15px 30px; background:#2196F3; color:white; border:none; border-radius:8px; cursor:pointer; font-weight:bold;">START TRACING</button>
    </div>

    <div id="tracer-ui" style="display:none;">
        <h2 id="title">Tracing Family A</h2>
        <p id="instr">Click points for an arm. Hit <b>'NEXT ARM'</b> for the next spiral.</p>
        <canvas id="mainCanvas" style="border:3px solid #333; cursor:crosshair; max-width:100%; background:#fff; margin-bottom:15px;"></canvas>
        <div>
            <button id="btnNext" style="padding:12px 20px; background:#2196F3; color:white; border:none; border-radius:8px; font-weight:bold;">NEXT ARM</button>
            <button id="btnUndo" style="padding:12px 20px; background:#9e9e9e; color:white; border:none; border-radius:8px; margin-left:10px;">UNDO POINT</button>
            <button id="btnFinish" style="padding:12px 20px; background:#4CAF50; color:white; border:none; border-radius:8px; margin-left:10px; font-weight:bold;">FINISH FAMILY A</button>
        </div>
    </div>
</div>

<script>
    let currentFamily = "A";
    let familyAData = [];
    let currentFamilyData = [[]];
    const fibs = [1, 2, 3, 5, 8, 13, 21, 34, 55, 89];

    const canvas = document.getElementById('mainCanvas');
    const ctx = canvas.getContext('2d');
    const img = new Image();

    function startTracing() {{
        if(!document.getElementById('name').value || !document.getElementById('email').value) {{ alert("Name and Email are required."); return; }}
        document.getElementById('info-form').style.display = 'none';
        document.getElementById('tracer-ui').style.display = 'block';
        img.src = "data:image/jpeg;base64,{img_base64}";
    }}

    img.onload = () => {{ canvas.width = img.width; canvas.height = img.height; render(); }};

    function render() {{
        ctx.clearRect(0, 0, canvas.width, canvas.height);
        ctx.drawImage(img, 0, 0);

        // Draw Family A in THICK BLACK for anchoring when on Family B
        if(currentFamily === "B") {{
            familyAData.forEach(arm => drawArm(arm, '#000000', 6, 12));
        }}

        // Draw Current Family (colors change per arm)
        currentFamilyData.forEach((arm, idx) => drawArm(arm, ['#FF0000', '#00FF00', '#0000FF', '#FFA500', '#800080'][idx % 5], 6, 12));
    }}

    function drawArm(arm, color, lineW, dotR) {{
        if(arm.length === 0) return;
        ctx.strokeStyle = color; ctx.fillStyle = color; ctx.lineWidth = lineW;
        ctx.beginPath();
        arm.forEach((pt, i) => {{
            if(i === 0) ctx.moveTo(pt[0], pt[1]); else ctx.lineTo(pt[0], pt[1]);
            ctx.moveTo(pt[0] + dotR, pt[1]); ctx.arc(pt[0], pt[1], dotR, 0, 2*Math.PI);
        }});
        ctx.stroke(); ctx.fill();
    }}

    canvas.onclick = (e) => {{
        const rect = canvas.getBoundingClientRect();
        const x = (e.clientX - rect.left) * (canvas.width / rect.width);
        const y = (e.clientY - rect.top) * (canvas.height / rect.height);
        currentFamilyData[currentFamilyData.length - 1].push([x, y]);
        render();
    }};

    document.getElementById('btnNext').onclick = () => {{ if(currentFamilyData[currentFamilyData.length-1].length>0) currentFamilyData.push([]); }};
    document.getElementById('btnUndo').onclick = () => {{
        let curr = currentFamilyData[currentFamilyData.length - 1];
        if(curr.length > 0) curr.pop(); else if(currentFamilyData.length > 1) currentFamilyData.pop();
        render();
    }};

    document.getElementById('btnFinish').onclick = () => {{
        const finalSet = currentFamilyData.filter(a => a.length > 0);
        const count = finalSet.length;

        if (currentFamily === "A") {{
            if (!fibs.includes(count)) {{
                alert("Family A has " + count + " arms. This is not a Fibonacci number! Please check your work.");
                return;
            }}
            familyAData = finalSet;
            currentFamily = "B";
            currentFamilyData = [[]];
            document.getElementById('title').innerText = "Tracing Family B";
            document.getElementById('instr').innerText = "Anchors (Family A) are in black. Click the intersections!";
            document.getElementById('btnFinish').innerText = "FINISH & DOWNLOAD ZIP";
            render();
        }} else {{
            const idxA = fibs.indexOf(familyAData.length);
            const prevFib = fibs[idxA - 1], nextFib = fibs[idxA + 1];

            if (count !== prevFib && count !== nextFib) {{
                alert("Family B has " + count + " arms. Since A is " + familyAData.length + ", B must be " + prevFib + " or " + nextFib + ".");
                return;
            }}

            const payload = JSON.stringify({{
                familyA: familyAData, familyB: finalSet,
                info: {{ name: document.getElementById('name').value, inst: document.getElementById('inst').value,
                        loc: document.getElementById('loc').value, email: document.getElementById('email').value }}
            }});
            google.colab.kernel.invokeFunction('notebook.FinalizeCollection', [payload], {{}});
            document.getElementById('tracer-ui').innerHTML = "<h2>Processing...</h2>";
        }}
    }};
</script>
"""
display(HTML(html_code))